In [11]:
import os

In [12]:
%pwd

'd:\\Projects\\End_to_End_MLProject_with-MLFLOW'

In [13]:
os.chdir("../")

In [14]:
%pwd

'd:\\Projects'

In [15]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir = Path
    source_URL = str
    local_data_file = Path
    unzip_dir = Path

In [16]:
from src.MLProject_With_MLFLOW.constants import *
from src.MLProject_With_MLFLOW.utils.common import read_yaml,create_directory

In [17]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
                

                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schema = read_yaml(schema_filepath)

                create_directory([self.config.artifacts_root])

    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directory([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                    root_dir = config.root_dir,
                    source_URL = config.source_URL,
                    local_data_file = config.local_data_file,
                    unzip_dir = config.unzip_dir                    
            )

            return data_ingestion_config

                



In [18]:
import os
import urllib.request as request
import zipfile
from src.MLProject_With_MLFLOW import logger
from src.MLProject_With_MLFLOW.utils.common import get_size


In [19]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config


    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} is download with header:\n{header}")
        else:
            logger.info(f"file is already exist or downloaded")

    def unzip_data(self):
        """unzip the downloaded data"""


        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'